In [1]:
# import useful libraries 
import pickle
from transformers import *

In [2]:
import torch

In [3]:
import util.util as u

In [4]:
filename = "/data/medg/misc/phuongpm/" + "sample1.0.json"

In [5]:
sample = u.load_json(filename)

In [6]:
pretrained = '../biobert_v1.1_pubmed'

In [47]:
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [9]:
data = u.JsonDataset(filename)

In [10]:
sample_data = list(data.json_to_plain(remove_notfound=False))

In [11]:
input0 = sample_data[0]

In [12]:
input0

{'c': ['blood investigations',
  'apgar score',
  'amniotic band syndrome',
  'myriad of deformities of fetal body parts',
  'distorted hemicranium',
  'adhesions',
  'head circumference',
  'craniofacial defect',
  'vascular disruption sequence',
  'minor craniofacial defects',
  'saggital suture',
  'significant vascular compromise',
  'shrivelled amniotic strands ) , ( b ) amniotic adhesions',
  'caput succedaneum',
  'concavity of the right parietal bone',
  'isolated cranial distortion mimicking caput succedenum',
  'blood sugar',
  'multiple congenital anomalies',
  'constrictive tissue bands',
  'multiple surgical procedures',
  'routine chromosome study',
  'syndrome',
  'intravenous cefotaxime',
  'treatment',
  'musculoskeletal and neurological examination of the baby',
  'relative preservation on the right side',
  'ear abnormality',
  'deformation',
  'dislocation of craniofacial structures',
  'mimicking caput succedaneum',
  'distortion',
  'antenatal scans',
  'fibrous b

In [13]:
text = input0['p']
#+ input0['q'].replace('▶ @placeholder', '[MASK]') + ' [SEP]'

In [14]:
tokenized_text = tokenizer.tokenize(text)

In [15]:
len(tokenized_text)

1301

In [16]:
tokenized_text

['isolated',
 'cr',
 '##anial',
 'distortion',
 'mimic',
 '##king',
 'cap',
 '##ut',
 'su',
 '##cc',
 '##ede',
 '##num',
 'from',
 'am',
 '##nio',
 '##tic',
 'band',
 'disruption',
 'without',
 'any',
 'neurological',
 'abnormal',
 '##ity',
 'summary',
 'this',
 'report',
 'describes',
 'a',
 'term',
 'newborn',
 'with',
 'isolated',
 'distortion',
 'in',
 'the',
 'left',
 'par',
 '##ie',
 '##tal',
 'bone',
 'without',
 'any',
 'other',
 'visible',
 'congenital',
 'anomaly',
 ',',
 'due',
 'to',
 'am',
 '##nio',
 '##tic',
 'band',
 'disruption',
 '.',
 'a',
 'skull',
 'x',
 '-',
 'ray',
 ',',
 'ultrasound',
 'scan',
 'and',
 'subsequent',
 'mri',
 'scan',
 'of',
 'the',
 'brain',
 'did',
 'not',
 'show',
 'any',
 'apparent',
 'distortion',
 'apart',
 'from',
 'depression',
 'and',
 'con',
 '##ca',
 '##vity',
 'in',
 'the',
 'left',
 'par',
 '##ie',
 '##tal',
 'bone',
 '.',
 'the',
 'purpose',
 'of',
 'this',
 'case',
 'report',
 'is',
 'to',
 'raise',
 'awareness',
 'of',
 'this',
 'po

In [17]:
query_toks = tokenizer.tokenize(input0['q'].replace('@placeholder', '[MASK]') + ' [SEP]')

In [18]:
query_toks

['[UNK]',
 '[MASK]',
 'from',
 'am',
 '##nio',
 '##tic',
 'band',
 'disruption',
 'is',
 'a',
 'possibility',
 '.',
 '[SEP]']

In [19]:
doc_max_len=tokenizer.max_len - len(query_toks) - 2 #[CLS] and [SEP]

In [20]:
doc_max_len

497

In [21]:
def getIndexPositions(listOfElements, element):
    ''' Returns the indexes of all occurrences of give element in
    the list- listOfElements '''
    indexPosList = []
    indexPos = 0
    while True:
        try:
            # Search for item in list from indexPos to the end of list
            indexPos = listOfElements.index(element, indexPos)
            # Add the index position in list
            indexPosList.append(indexPos)
            indexPos += 1
        except ValueError as e:
            break
 
    return indexPosList

In [22]:
# finding sentences in the passage
index_end_sent = getIndexPositions(tokenized_text, '.')

In [23]:
sent1 = ['[CLS]'] + tokenized_text[:480] + ['SEP']
sent2 = ['[CLS]'] + tokenized_text[480:971]+ ['SEP']
sent3 = ['[CLS]'] + tokenized_text[971:]+ ['SEP']

In [24]:
indexed_tokens = tokenizer.convert_tokens_to_ids(sent1 + query_toks)

In [25]:
segments_ids = [0]*len(sent1) + [1]*len(query_toks)

In [26]:
len(segments_ids) == len(sent1 + query_toks)

True

In [27]:
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [28]:
# tokens_tensor = tokens_tensor.to('cuda')
# segments_tensors = segments_tensors.to('cuda')
# model.to('cuda')

In [67]:
candidates = [tokenizer.tokenize(i) for i in input0['c']]
candidateids = [tokenizer.convert_tokens_to_ids(i) for i in candidates]
flatten_cands = []
for i in candidateids:
    flatten_cands.extend(i)
flatten = set(flatten_cands)

In [72]:
with torch.no_grad():
    outputs = model(tokens_tensor, token_type_ids=segments_tensors)
    predictions = outputs[0]

In [73]:
predicted_index = predictions[0, len(sent1) + 1, list(flatten)]
# predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]

In [70]:
answer_idx = 

tensor([ 2.5285e+00,  8.3576e-01, -2.3935e+00, -8.1137e-01,  7.6414e+00,
         3.8085e-01,  2.9014e+00,  1.5688e+00,  3.5203e+00,  6.3270e+00,
         1.8005e+00, -3.9622e+00,  1.6744e+00, -4.7843e+00, -1.0267e+00,
         2.5868e+00, -5.8010e+00,  5.2336e-01,  9.5994e-02,  2.2915e+00,
        -4.0275e+00, -1.7311e+00,  1.4442e+00, -2.4006e+00, -1.8713e+00,
        -5.1228e+00,  1.4464e+00,  1.6116e+00, -2.5458e+00,  1.9800e+00,
        -4.4311e+00,  1.1366e+00,  6.7778e-01, -2.5040e+00, -1.6833e+00,
        -7.9599e-01, -3.2413e-01,  3.0007e+00, -9.7884e-01,  7.4727e-01,
        -2.6176e+00,  8.1412e-01,  1.5103e+00,  7.4303e+00, -4.8734e+00,
         1.6225e+00,  1.1380e+00, -4.6375e-01,  4.9648e+00,  8.1434e-01,
        -1.1045e+00, -6.2286e+00,  3.8701e+00,  3.4522e-01, -7.7602e-01,
         4.7956e+00, -9.7767e-01,  5.1857e+00,  4.7559e+00,  2.0051e+00,
        -9.4941e-01, -5.5339e+00, -6.9401e+00, -3.7441e+00, -2.6547e+00,
         1.4493e+00, -2.1867e+00, -8.9202e-01,  7.6

In [62]:
predictions

tensor([[[ -8.6872,  -8.6604,  -8.6337,  ...,  -8.5176,  -7.6843,  -5.3905],
         [ -6.4908,  -6.3169,  -6.4961,  ...,  -6.1162,  -6.0905,  -5.9012],
         [-13.3816, -14.1228, -13.6414,  ..., -12.2075, -10.2132,  -8.8761],
         ...,
         [ -6.2822,  -6.5075,  -6.6013,  ...,  -5.5398,  -5.3947,  -8.0459],
         [-13.7735, -13.7491, -13.9186,  ..., -12.3411, -12.6793,  -9.6002],
         [-16.1311, -16.3270, -16.3317,  ..., -16.5506, -14.1881,  -9.2726]]])